* Checkout new git branch for changes
* Update user detail HTML page
* Update user form HTML page
* Review logic to update data using flask shell
* Add update user logic to the application
* Run application and validate
* Validate CI/CD Pipeline using GitHub Actions
* Exercise and Solution

* Checkout new git branch for changes
  * Use command `git checkout -b feature/crudupdate`

* Update user detail HTML page

```html
{%- extends "base.html" %}

{% block content %}
    <h3>User Details</h3>
    <div class="container">
        <form method="GET" action="{{ url_for('user', id=user.id) }}">
            <input type="text" name="id" value="{{ user.id }}" hidden /> 
            <label for="first_name"><b>First Name</b></label>
            {{ user.first_name }}
            <br><br>
            <label for="last_name"><b>Last Name</b></label>
            {{ user.last_name }}
            <br><br>
            <label for="username"><b>Username</b></label>
            {{ user.username }}
            <br><br>
            <label for="email"><b>Email</b></label>
            {{ user.email }}
            <br><br>

            <button type="submit" name="action" value="edit">Edit</button>
        </form>
    </div>
{%- endblock %}
```

* Update user form HTML page

Add id as hidden field. Also add logic to populate fields when edit is clicked on user detail page.
```html
{%- extends "base.html" %}

{% block content %}
    <h3>User Form</h3>
    <div class="container">
        <form method="POST" action="{{ url_for('user') }}">
            <input type="text" name="id" value="{{ user.id }}" hidden /> 
            <label for="first_name"><b>First Name</b></label>
            <input type="text" placeholder="Enter First Name" name="first_name" value="{{ user.first_name}}"/>
            <br><br>
            <label for="last_name"><b>Last Name</b></label>
            <input type="text" placeholder="Enter Last Name" name="last_name" value="{{ user.last_name }}"/>
            <br><br>
            <label for="username"><b>Username</b></label>
            <input type="text" placeholder="Enter Username" name="username" value="{{ user.username }}" required />
            <br><br>
            <label for="email"><b>Email</b></label>
            <input type="text" placeholder="Enter Email Id" name="email" value="{{ user.email }}"/>
            <br><br>

            <button type="submit">Save</button>
        </form>
    </div>
{%- endblock %}
```

* Review logic to update data using flask shell

Launch flask shell using `python -m flask shell` and run the below code snippets.
```python
from models.user import User
u = User.query.get(1)
u.first_name # Read existing value
u.first_name = 'Scotty'
db.session.commit() # Updates the data in the database table

u = User.query.get(1)
u.first_name # Should return Scotty
```

Use pgAdmin and run the following query to confirm.
```sql
SELECT * FROM users WHERE id = 1;
```

* Add update user logic to the application

Update `app.py` with the below logic. The method will be invoked using POST when edit is clicked on user details page. It includes the logic of insert as well as update.

```python
@app.route('/user', methods=['GET', 'POST'])
def user():
    if request.method == 'GET':
        id = request.args.get('id')
        if id:
            user = User.query.get(id)
            form_action = request.args.get('action')
            if form_action == 'edit':
                return render_template('user_form.html', user=user)
            return render_template('user_detail.html', user=user)
        else:
            return render_template('user_form.html')
    elif request.method == 'POST':
        id = request.form['id']
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        username = request.form['username']
        email = request.form['email']
        if id:
            user = User.query.get(id)
            user.first_name = first_name
            user.last_name = last_name
            user.username = username
            user.email = email
        else:
            user = User(
                first_name=first_name, 
                last_name=last_name, 
                username=username,
                email=email
            )
            db.session.add(user)
        db.session.commit()
        return redirect(url_for('users'))
```

* Run application and validate

Review the following details before running
1. Changes to user detail page
2. Changes to user form page
3. Changes to application to insert or update the data.

Run the application using Flask.

```shell
python -m flask run
```

Here are the steps to validate the application.
1. Visit Users Dashboard http://localhost:5000/users
2. Visit User Details page for 1 - http://localhost:5000/users?id=1
3. Click on edit and update the first name to "Scotty". Submit the changes by clicking on Save.
4. The page will be redirected to users page. Make sure to review and confirm if the first name is changed or not.


* Validate CI/CD Pipeline using GitHub Actions

1. Push changes to remote repository
2. Validate GitHub Action or CI/CD Pipeline
3. Validate Application in GCP VM

* Exercise - Develop functionality to update course details to the table.

1. Update course detail html page with **Edit** button. You also need to convert to form so that the data can be submitted.
2. Update course form html page to display the details of course to be edited.
3. Update function `course` for `/course` end point with `POST` to update data in the table.
4. Run and Validate the application. Go to the end point `/course` with `?course_id=1` parameter and perform below steps.
  * Make sure to click on **Edit** button.
  * Confirm if the course details are displayed in the form or not.
  * Click on **Save** and make sure data is saved into the table.
  * Also, the page should be redirected to **courses** dashboard. The record edited should have updated data.
5. Make sure to merge the changes to GitHub main branch and deploy the changes to Remote VM. In case, if you want you can validate as well.

* Solution - Develop functionality to update course details in the table.

Add `templates/course_detail.html`

```html
{%- extends "base.html" %}

{% block content %}
    <h3>Course Details</h3>
    <div class="container">
        <form method="GET" action="{{ url_for('course') }}">
            <input type="text" name="course_id" value="{{ course.course_id }}" hidden /> 
            <label for="course_name"><b>Course Name</b></label>
            {{ course.course_name }}
            <br><br>
            <label for="course_author"><b>Course Author</b></label>
            {{ course.course_author }}
            <br><br>
            <label for="course_endpoint"><b>Course Endpoint</b></label>
            {{ course.course_endpoint }}
            <br><br>

            <button type="submit" name="action" value="edit">Edit</button>
        </form>
    </div>
{%- endblock %}
```

Update `templates/course_form.html`

```html
{%- extends "base.html" %} 

{% block content %}
    <h3>Course Form</h3>
    <div class="container">
        <form method="POST" action="{{ url_for('course') }}">
            <input type="text" name="course_id" value="{{ course.course_id }}" hidden /> 
            <label for="course_name"><b>Course Name</b></label>
            <input type="text" placeholder="Enter Course Name" name="course_name" value="{{ course.course_name}}" />
            <br><br>
            <label for="course_author"><b>Course Author</b></label>
            <input type="text" placeholder="Enter Course Author Name" name="course_author" value="{{ course.course_author}}" />
            <br><br>
            <label for="course_endpoint"><b>Course Endpoint</b></label>
            <input type="text" placeholder="Enter Course Endpoint" name="course_endpoint" value="{{ course.course_name}}" required />
            <br><br>

            <button type="submit">Save</button>
        </form>
    </div>
{%- endblock %}
```

Update functionality in `course` function of `app.py`

```python
@app.route('/course', methods=['GET', 'POST'])
def course():
    if request.method == 'GET':
        course_id = request.args.get('course_id')
        if course_id:
            course = Course.query.get(course_id)
            form_action = request.args.get('action')
            if form_action == 'edit':
                return render_template('course_form.html', course=course)
            return render_template('course_detail.html', course=course)
        else:
            return render_template('course_form.html', course=None)
    elif request.method == 'POST':
        course_id = request.form['course_id']
        course_name = request.form['course_name']
        course_author = request.form['course_author']
        course_endpoint = request.form['course_endpoint']
        if course_id:
            course = Course.query.get(course_id)
            course.course_name = course_name
            course.course_author = course_author
            course.course_endpoint = course_endpoint
        else:
            course = Course(
                course_name=course_name, 
                course_author=course_author, 
                course_endpoint=course_endpoint
            )
            db.session.add(course)
        db.session.commit()
        return redirect(url_for('courses'))
```

Restart the application and then go to `http://localhost:5000/course?course_id=1`. Update details for course id 1 and click on Save. You will be redirected to `http://localhost:5000/courses`.
* Course Name: Mastering Python
* Course Author: Durga Gadiraju
* Course Endpoint: mastering-python

Also go to `http://localhost:5000/course?course_id=1` to see the updated course details. Feel free to change the course_id parameter value if it is different from 1.
